In [ ]:
!pip install conllu

In [ ]:
from conllu import parse_incr
from io import open
file=open('hi_hdtb-ud-train.conllu','r',encoding='utf-8')
ud_files=[]
for tokenlist in parse_incr(file):
    ud_files.append(tokenlist)

In [ ]:
def dataset(ud_files):
    bank=[]
    for sentence in ud_files:
        tokens=[]
        tags=[]

        for token in sentence:
            tokens.append(token['form'])
            tags.append(token['upostag'])

        bank.append((tokens,tags))
    return bank

In [ ]:
train=dataset(ud_files)

In [ ]:
def separate(bank):
    X,y=[],[]
    for index in range(len(bank)):
        X.append(bank[index][0])
        y.append(bank[index][1])
    return X,y

In [ ]:
X,y=separate(train)

In [ ]:
file=open('hi_hdtb-ud-test.conllu','r',encoding='utf-8')
ud_files=[]
for tokenlist in parse_incr(file):
    ud_files.append(tokenlist)
test = dataset(ud_files)

In [ ]:
file=open('hi_hdtb-ud-dev.conllu','r',encoding='utf-8')
ud_files=[]
for tokenlist in parse_incr(file):
    ud_files.append(tokenlist)
dev = dataset(ud_files)

In [ ]:
Xtest,ytest=separate(test)

In [ ]:
Xdev,ydev=separate(dev)

In [ ]:
tag_list = set()
tag_count = {}
word_set = set()

def transition_count(X,y): ## Function for calculating the frequency of tag combinations.
    global tag_list # example: Noun-verb , noun-adjective, verb-adjective etc.
    global word_set
    transition_dict = {} # initialize dictionary containing tag combinations as key and value as frequency.
    global tag_count
    for v in range(len(X)):
        previous="start"
        for data in range(len(X[v])):
            i=X[v][data]
            word = i
            word_set.add(word.lower())
            tag = y[v][data]
            tag_list.add(tag)

            if tag in tag_count:
                tag_count[tag]+=1
            else:
                tag_count[tag] = 1


            if (previous + "~tag~" + tag) in transition_dict: #if tag combination is already present then increment count by 1
                    transition_dict[previous + "~tag~" + tag] += 1
                    previous = tag
            else:  # if tag combination not present then initialize count for that combination to 1.
                    transition_dict[previous + "~tag~" + tag] = 1
                    previous = tag

    return transition_dict,tag_count,tag_list,word_set  

In [ ]:
transmission_m,tag_count,tag_list,word_set = transition_count(X,y) 

In [ ]:
def transition_probability(X,y):
    #count_dict = transition_count(X,y)
    count_dict = transmission_m
    prob_dict = {}
    for key in count_dict:
        den = 0
        val = key.split("~tag~")[0]
        # Probabilty of a tagA to be followed by tagB out of all possible tags # 
        for key_2 in count_dict:
            if key_2.split("~tag~")[0] == val:
                den += count_dict[key_2]
        prob_dict[key] = Decimal(count_dict[key])/(den)
    return prob_dict

In [ ]:
def transition_smoothing(X,y):
    transition_prob = transition_probability(X,y)
    for tag in tag_list:
        # if a tag does not occur as a start tag, then set its probability to be a start tag to minimum value #
        if "start" + tag not in  transition_prob:
            transition_prob[("start" + "~tag~" + tag)] = Decimal(1) / Decimal(len(word_set) + tag_count[tag])
    for tag1 in tag_list:
        for tag2 in tag_list:
        # if a particular tag combination does not exist in the dictionary, we set its probability to minimum#
            if (tag1 +"~tag~" + tag2) not in transition_prob:
                transition_prob[(tag1+"~tag~"+tag2)] = Decimal(1)/Decimal(len(word_set) + tag_count[tag1])
    return transition_prob

In [ ]:
def emission_count(X,y):   #FUNCTION FOR MAPPING THE WORDS WITH CORRESPONDING TAGS AND CALCULATE FREQUENCY OF EACH WORD AND TAG COMBINATION
    count_word = {}
    for v in range(len(X)):
        for data in range(len(X[v])):
            i = X[v][data]
            word = i
            tag = y[v][data]
            # map the words in the training set to their tagged POS #
            if word.lower() + "/" + tag in count_word:
                count_word[word.lower() + "/" + tag] +=1
            else:
                count_word[word.lower() + "/" + tag] = 1
    return count_word  #RETURN DICTIONARY CONATINING THE EMISSION COUNT

In [ ]:
def emission_probability(X,y):
    global tag_count
    word_count = emission_count(X,y)
    emission_prob_dict = {}
    # calculate probability of a word to be a certain Tag out of all the possible tags that it can be #
    for key in word_count:
        emission_prob_dict[key] = Decimal(word_count[key])/tag_count[key.split("/")[-1]]
    return emission_prob_dict

In [ ]:
from decimal import *

In [ ]:
transition_model = transition_smoothing(X,y)
emission_model = emission_probability(X,y)

In [ ]:
def viterbi_algorithm(sentence, tag_list, transition_prob, emission_prob,tag_count, word_set):
    global tag_set
    # Get words from each sentence 
    sentence = sentence.strip("\n")
    word_list = sentence.split(" ")
    current_prob = {}
    for tag in tag_list:
        # transition probability set to minimum
        tp = Decimal(0)
        # Emission probability set to minimum
        em = Decimal(0)
        # Storing the probability of every tag to be starting tag #
        if "start~tag~"+tag in transition_prob:
            tp = Decimal(transition_prob["start~tag~"+tag])
        # Check for word in training data. If present, check the probability of the first word to be of given tag#
        if word_list[0].lower() in word_set:
            if (word_list[0].lower()+"/"+tag) in emission_prob:
                em = Decimal(emission_prob[word_list[0].lower()+"/"+tag])
                # Storing probability of current combination of tp and em #
                current_prob[tag] = tp * em
         # Check for word in training data. If absent then probability is just tp# 
        else:
            em = Decimal(1) /(tag_count[tag] +len(word_set))
            current_prob[tag] = tp

    if len(word_list) == 1:
        # Return max path if only one word in sentence #
        max_path = max(current_prob, key=current_prob.get)
        return max_path
    else:
        # Tracking from second word to last word #
        for i in range(1, len(word_list)):
            previous_prob = current_prob
            current_prob = {}
            locals()['dict{}'.format(i)] = {}
            previous_tag = ""
            for tag in tag_list:
                if word_list[i].lower() in word_set:
                    if word_list[i].lower()+"/"+tag in emission_prob:
                        em = Decimal(emission_prob[word_list[i].lower()+"/"+tag])
                        # Find the maximum probability using previous node's(tp*em)[i.e probability of reaching to the previous node] * tp * em (Bigram Model) #
                        max_prob, previous_state = max((Decimal(previous_prob[previous_tag]) * Decimal(transition_prob[previous_tag + "~tag~" + tag]) * em, previous_tag) for previous_tag in previous_prob)
                        current_prob[tag] = max_prob
                        locals()['dict{}'.format(i)][previous_state + "~" + tag] = max_prob
                        previous_tag = previous_state
                else:
                    em = Decimal(1) /(tag_count[tag] +len(word_set))
                    max_prob, previous_state = max((Decimal(previous_prob[previous_tag]) * Decimal(transition_prob[previous_tag+"~tag~"+tag]) * em, previous_tag) for previous_tag in previous_prob)
                    current_prob[tag] = max_prob
                    locals()['dict{}'.format(i)][previous_state + "~" + tag] = max_prob
                    previous_tag = previous_state

            # if last word of sentence, then return path dicts of all words #
            if i == len(word_list)-1:
                max_path = ""
                last_tag = max(current_prob, key=current_prob.get)
                max_path = max_path + last_tag + " " + previous_tag
                for j in range(len(word_list)-1,0,-1):
                    for key in locals()['dict{}'.format(j)]:
                        data = key.split("~")
                        if data[-1] == previous_tag:
                            max_path = max_path + " " +data[0]
                            previous_tag = data[0]
                            break
                result = max_path.split()
                result.reverse()
                return " ".join(result)

In [ ]:
sent='यह एशिया की सबसे बड़ी मस्जिदों में से एक है ।'
path = viterbi_algorithm(sent, tag_list, transition_model, emission_model,tag_count, word_set)
word = sent.split(" ")
tag = path.split(" ")
for j in range(0,len(word)):
    if(j==len(word)-1):
        print(word[j] + "/" + tag[j]+ u'\n')
    else:
        print(word[j] + "/" + tag[j] + " ")